In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/tarasbohdan/Desktop/Experiments-AAAI-24


## Import dependencies

In [4]:
import os
import copy
import numpy as np

from virny.utils.custom_initializers import create_config_obj

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

In [5]:
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

In [6]:
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [7]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_student_performance'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [
    "Ax=50, Az=10, Ay=10",
    "Ax=10, Az=50, Ay=10",
    "Ax=50, Az=50, Ay=10"
]
# FAIR_INTERVENTION_PARAMS_LST = [
#     0.0, 0.1, 0.2, 0.3, 0.4
# ]
config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'student_performance_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [8]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'dmytro_experiments'

In [9]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [10]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  012d073d-00aa-4cd1-b3fe-5545796c43ad


## Initialize custom objects

In [11]:
class Config:
  file_name = './data/student_performance/student-mat.csv'

In [12]:
# from notebooks.mult_repair_levels_student_performance.StudentPerformanceDataset import StudentPerformanceDataset
#
# data_loader = StudentPerformanceDataset(Config.file_name)
# data_loader.X_data.head()

In [13]:
from virny.datasets import ACSIncomeDataset

In [14]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False,
                               subsample_size=500, subsample_seed=42)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,23,7,3,230,36,0,1,1,55,55.0
1,16,1,5,4110,13,2,2,1,20,35.0
2,16,4,3,4130,51,0,2,1,59,30.0
3,18,4,1,4020,13,0,1,2,43,40.0
4,14,1,1,8300,20,1,2,2,33,20.0


In [17]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [18]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-11-20 16:22:41 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': "['Ax=50, Az=10, Ay=10', 'Ax=10, Az=50, "
                                 "Ay=10', 'Ax=50, Az=50, Ay=10']",
 'intervention_param': 'Ax=50, Az=50, Ay=10',
 'model_init_seed': 400,
 'session_uuid': '012d073d-00aa-4cd1-b3fe-5545796c43ad'}




2023-11-20 16:22:41 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 74, 278, 209, 121, 361,  67, 289,  80,  54, 261, 134, 304, 222,
             88, 266, 466, 183, 180, 328, 439],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 74, 278, 209, 121, 361,  67, 289,  80,  54, 261, 134, 304, 222,
             88, 266, 466, 183, 180, 328, 439],
           dtype='int64')


Multiple alphas:   0%|          | 0/3 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

[{'metric': 'Mean', 'model_name': 'RandomForestClassifier', 'model_params': "{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 400, 'verbose': 0, 'warm_start': False}", 'dataset_name': 'StudentPerformance', 'num_estimators': 10, 'tag': 'OK', 'record_create_date_time': Timestamp('2023-11-20 14:23:01.795233+0000', tz='UTC'), 'session_uuid': '012d073d-00aa-4cd1-b3fe-5545796c43ad', 'experiment_iteration': 'Exp_iter_4', 'dataset_split_seed': 400, 'model_init_seed': 400, 'fair_intervention_params_lst': "['Ax=50, Az=10, Ay=10', 'Ax=10, Az=50, Ay=10', 'Ax=50, Az=50, Ay=10']", 'intervention_param': 'Ax=10, Az=50, Ay=10', 'subgroup': 'SEX_priv', 'metric_value': 1.0}, {'metric': 'Std', 'model_name': 

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

[{'metric': 'Mean', 'model_name': 'RandomForestClassifier', 'model_params': "{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 400, 'verbose': 0, 'warm_start': False}", 'dataset_name': 'StudentPerformance', 'num_estimators': 10, 'tag': 'OK', 'record_create_date_time': Timestamp('2023-11-20 14:23:09.504358+0000', tz='UTC'), 'session_uuid': '012d073d-00aa-4cd1-b3fe-5545796c43ad', 'experiment_iteration': 'Exp_iter_4', 'dataset_split_seed': 400, 'model_init_seed': 400, 'fair_intervention_params_lst': "['Ax=50, Az=10, Ay=10', 'Ax=10, Az=50, Ay=10', 'Ax=50, Az=50, Ay=10']", 'intervention_param': 'Ax=50, Az=50, Ay=10', 'subgroup': 'SEX_priv', 'metric_value': 1.0}, {'metric': 'Std', 'model_name': 